<a href="https://colab.research.google.com/github/nazbeh/I_C_M_E_2020/blob/master/Workshop4/MPI_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MPI Tutorial 

## Hello World

In [ ]:
%%file mpi_hello_world.c
#include <stdio.h>
#include "mpi.h"

int main(int argc, char **argv){ 

    MPI_Init(&argc, &argv);

    int rank, size;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    printf("Hello from processor  %d out of  %d\n", rank, size); 
 
    MPI_Finalize(); 
} 

Compile using ```mpicc``` 

In [ ]:
!mpicc -o mpi_hello_world mpi_hello_world.c

Execute using ```mpirun```and specify the number of ranks using ```-n```. 

Note: Do no use the flag ```--allow-run-as-root```in your personal applications. This is a particular case for Google Colaboratory

In [ ]:
!mpirun -n 4 --allow-run-as-root ./mpi_hello_world

## Example: Compute Pi

In [ ]:
%%file pi_mpi.c
#include <stdio.h>
#include "mpi.h"
int main(int argc, char **argv){
    MPI_Init(&argc,&argv);
    int rank,size;
    MPI_Comm_rank(MPI_COMM_WORLD,&rank);
    MPI_Comm_size(MPI_COMM_WORLD,&size);

    int n = 10000;
    double total;
    double s = 0.;
    for (int i = rank; i< n; i+= size){
        s+= 4./ (1.+(i+0.5)*(i+0.5)/(double)(n*n))*1./(double) n;
    }
    MPI_Reduce(&s,&total,1,MPI_DOUBLE,MPI_SUM,0,MPI_COMM_WORLD);
    if (rank == 0){
        printf("Pi is %f\n",total);
    }
    MPI_Finalize();
}

In [ ]:
!mpicc -o pi_mpi pi_mpi.c

In [ ]:
!mpirun -n 4 --allow-run-as-root ./pi_mpi

# Example: Laplacian MPI

### Sequential Version

In [ ]:
%%file laplacian.cpp

#include <math.h>
#include <stdio.h>
#include <stdlib.h>

int main(int argc, char* argv[])
{
    const int N1 = 1024;
    const int N2 = 1024;
    const int ITER = 100;
    const int EPS = 0.1;

    double* a = (double*) malloc((N1 * N2)*sizeof(double));
    double* b = (double*) malloc((N1 * N2)*sizeof(double));
    double x[N1];
    double y[N2];
    
    int row;

    //Initialize arrays x and y
    for(int i = 0; i < N1; ++i)
        x[i] = i / (double)(N1 - 1);
    
    for(int j = 0; j < N2; ++j)
        y[j] = j / (double)(N2 - 1);


    //Initialize a and b
    for(int i = 0 ; i < N1; ++i){
        row = i * N2;
        if (x[i] <= 0.5){
             for(int j = 0; j < N2; ++j){
                a[row + j] = cos(x[i] + y[j]);
                b[row + j] = a[row + j];}}
        else{
            for(int j = 0; j < N2; ++j){
                a[row + j] = sin(x[i] + y[j]);
                b[row + j] = a[row + j];}}}

    
    for(int n = 0; n < ITER; ++n){
        //Laplacian smoothing
        for(int i = 1; i < (N1 - 1); ++i){
            row = i * N2;
            for(int j = 1; j < (N2 - 1); ++j){
                a[row + j] = b[row + j] + EPS * (b[row + j - 1] +
                             b[row + j + 1] - 8.0 * b[row + j] + b[row + j - N2] +
                             b[row + j - N2 - 1] + b[row + j - N2 + 1] + 
                             b[row + j + N2] + b[row + j + N2 + 1] + b[row + j + N2 -1]);   
            }
        }

        //Copy a into b
        for(int i = 1; i < (N1 - 1); ++i){
            row = i * N2;
            for(int j = 1; j < (N2 - 1); ++j)
                b[row + j] = a[row + j];
        }
    }
    double average_temperature = 0;
    for(int i = 1; i < (N1 - 1); ++i){
       row = i * N2;
       for(int j = 1; j < (N2 - 1); ++j)
          average_temperature += b[row + j];}
    average_temperature = average_temperature/(double)((N1-2) * (N2-2));
 
    printf("average temp = %.3f\n",average_temperature);  

    free(a);
    free(b);
}

In [ ]:
!g++ -o laplacian laplacian.cpp

In [ ]:
!./laplacian

### MPI version 

In [ ]:
%%file laplace_mpi.c

#include "mpi.h"
#include <math.h>
#include <stdio.h>
#include <stdlib.h>

int main(int argc, char* argv[])
{
    MPI_Init(&argc, &argv);
    
    double start,finish,time;
    MPI_Barrier(MPI_COMM_WORLD);
    start = MPI_Wtime();
 
    //Problem parameters 
    const int  N1= 1024;
    const int  N2= 1024;
    const int  ITER = 100;
    const double  EPS= 0.1;
    
    //Get identification wrt other ranks
    int rank, size;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);
 
    //Define number of elements
    int n1;
    int n2 = N2;
    if (rank == 0){
      n1 = N1/size + 1;}
    else if (rank < (size-1)){
      n1 = N1/size + 2;}
    else{
      n1 = N1 - size*(N1/size) + 1;}
 
    //Initial coordinate in x
    int i_0 = rank * (n1 - 2) - 1 ;
 
   //Initial coordinate in y
    int j_0 = 0;
    int row;

    //Define procesors in boundaries
    int up_bd = rank-1;
    int down_bd = rank+1;
 
    //Create sparse matrices
    double* a = (double*) malloc((n1 * n2)*sizeof(double));
    double* b = (double*) malloc((n1 * n2)*sizeof(double));
    double x[n1];
    double y[n2];
    
    

    //Initialize arrays x and y
    for(int i = 0; i < n1; ++i){
        x[i] = (i_0 + i) / (double)(N1 - 1);}
    
    for(int j = 0; j < n2; ++j){
        y[j] = (j_0 + j) / (double)(N2 - 1);}


    //Initialize a and b
    for(int i = 0 ; i < n1; ++i){
        row = i * n2;
        if (x[i] <= 0.5){
            for(int j = 0; j < n2; ++j){
                a[row + j] = cos(x[i] + y[j]);
                b[row + j] = a[row + j];}}
        else{
            for(int j = 0; j <n2; ++j){
                a[row + j] = sin(x[i] + y[j]);
                b[row + j] = a[row + j];}}}

    //Laplacian smoothing    
    MPI_Status status;
    for(int n = 0; n < ITER; ++n){
        //Laplacian smoothing
        for(int i = 1; i < (n1 - 1); ++i){
            row = i * n2;
            for(int j = 1; j < (n2 - 1); ++j){
                a[row + j] = b[row + j] + EPS * (b[row + j - 1] +
                             b[row + j + 1] - 8.0 * b[row + j] + b[row + j - n2] +
                             b[row + j - n2 - 1] + b[row + j - n2 + 1] + 
                             b[row + j + n2] + b[row + j + n2 + 1] + b[row + j + n2 -1]);}}   

        //Copy a into b
        for(int i = 1; i < (n1 - 1); ++i){
            row = i * n2;
            for(int j = 1; j < (n2 - 1); ++j)
                b[row + j] = a[row + j];}

        //Copy from other processors
        if (rank > 0){
            MPI_Send(&(b[n2]),n2,MPI_DOUBLE,up_bd,0,MPI_COMM_WORLD);
            MPI_Recv(&(b[0]),n2,MPI_DOUBLE,up_bd,0,MPI_COMM_WORLD,&status);}

        if (rank < (size-1)){
            MPI_Recv(&(b[(n1 - 1) * n2]),n2,MPI_DOUBLE,down_bd,0,MPI_COMM_WORLD,&status);
            MPI_Send(&(b[(n1 - 2) * n2]),n2,MPI_DOUBLE,down_bd,0,MPI_COMM_WORLD);}}
   
   
   free(a);
   free(b);  
   MPI_Barrier(MPI_COMM_WORLD);
   finish = MPI_Wtime();
   time = finish - start;
   printf("rank= %d, n1 = %d, time = %f\n",rank, n1, time);  
  
   MPI_Finalize();
      
}


In [ ]:
!mpicc -o  laplace_mpi laplace_mpi.c -lm

In [ ]:
!mpirun -n 10 --allow-run-as-root ./laplace_mpi

### Exercise: Adding an average temperature

In the previous code we forgot to compute the average temperature. In these case we want to compute the average temperature per rank and globally, complete the ```//To do```. Notice that you also need to complete some MPI code.

It may be usfeul to know that the syntax
```
MPI_Bcast(*buffer, count, mpi type, from rank, mpi communicator);
```
For more information visit: https://www.open-mpi.org/doc/current/


In [ ]:
%%file laplace_mpi.c
// To do here, add mpi header

#include <math.h>
#include <stdio.h>
#include <stdlib.h>

int main(int argc, char* argv[])
{
    // To do here, start MPI
    
    
    double start,finish,time;
    MPI_Barrier(MPI_COMM_WORLD);
    start = MPI_Wtime();
 
    //Problem parameters 
    const int  N1= 1024;
    const int  N2= 1024;
    const int  ITER = 100;
    const double  EPS= 0.1;
    
    //To do here: get identification wrt other ranks
    int rank, size;



    //Define number of elements
    int n1;
    int n2 = N2;
    if (rank == 0){
      n1 = N1/size + 1;}
    else if (rank < (size-1)){
      n1 = N1/size + 2;}
    else{
      n1 = N1 - size*(N1/size) + 1;}
 
    //Initial coordinate in x
    int i_0 = rank * (n1 - 2) - 1 ;
 
   //Initial coordinate in y
    int j_0 = 0;
    int row;

    //Define procesors in boundaries
    int up_bd = rank-1;
    int down_bd = rank+1;
 
    //Create sparse matrices
    double* a = (double*) malloc((n1 * n2)*sizeof(double));
    double* b = (double*) malloc((n1 * n2)*sizeof(double));
    double x[n1];
    double y[n2];
    
    

    //Initialize arrays x and y
    for(int i = 0; i < n1; ++i){
        x[i] = (i_0 + i) / (double)(N1 - 1);}
    
    for(int j = 0; j < n2; ++j){
        y[j] = (j_0 + j) / (double)(N2 - 1);}


    //Initialize a and b
    for(int i = 0 ; i < n1; ++i){
        row = i * n2;
        if (x[i] <= 0.5){
            for(int j = 0; j < n2; ++j){
                a[row + j] = cos(x[i] + y[j]);
                b[row + j] = a[row + j];}}
        else{
            for(int j = 0; j <n2; ++j){
                a[row + j] = sin(x[i] + y[j]);
                b[row + j] = a[row + j];}}}

    //Laplacian smoothing    
    MPI_Status status;
    for(int n = 0; n < ITER; ++n){
        //Laplacian smoothing
        for(int i = 1; i < (n1 - 1); ++i){
            row = i * n2;
            for(int j = 1; j < (n2 - 1); ++j){
                a[row + j] = b[row + j] + EPS * (b[row + j - 1] +
                             b[row + j + 1] - 8.0 * b[row + j] + b[row + j - n2] +
                             b[row + j - n2 - 1] + b[row + j - n2 + 1] + 
                             b[row + j + n2] + b[row + j + n2 + 1] + b[row + j + n2 -1]);}}   

        //Copy a into b
        for(int i = 1; i < (n1 - 1); ++i){
            row = i * n2;
            for(int j = 1; j < (n2 - 1); ++j)
                b[row + j] = a[row + j];}

        //Copy from other processors
        if (rank > 0){
            MPI_Send(&(b[n2]),n2,MPI_DOUBLE,up_bd,0,MPI_COMM_WORLD);
            MPI_Recv(&(b[0]),n2,MPI_DOUBLE,up_bd,0,MPI_COMM_WORLD,&status);}

        if (rank < (size-1)){
            MPI_Recv(&(b[(n1 - 1) * n2]),n2,MPI_DOUBLE,down_bd,0,MPI_COMM_WORLD,&status);
            MPI_Send(&(b[(n1 - 2) * n2]),n2,MPI_DOUBLE,down_bd,0,MPI_COMM_WORLD);}}
   
   //Get average temperature
 
   //Compute total sum temperature locally
   double sum_temperature = 0;
   for(int i = 1; i < (n1 - 1); ++i){
       row = i * n2;
       for(int j = 1; j < (n2 - 1); ++j)
          sum_temperature += b[row + j];}
   
   // To do here: Reduce all the sum of temperatures to rank 0
   double total_temperature;
   
   //Computes global average temperature
   double average_temperature;
   if (rank == 0){
       average_temperature = total_temperature/(double)((N1-2) * (N2-2));
   }
 
   // To do here: Broadcast to all the global average temperature from rank 0
   
   double local_temperature = sum_temperature/(double)((n2-2)*(n1-2));
   
   free(a);
   free(b);  
   MPI_Barrier(MPI_COMM_WORLD);
   finish = MPI_Wtime();
   time = finish - start;
   printf("rank= %d, n1 = %d, time = %f, global temp = %.3f, local temp = %.3f \n",rank, n1, time,average_temperature,local_temperature);  
  
   MPI_Finalize();
      
}

Once you are done, compile and execute the code

In [ ]:
!mpicc -o  laplace_mpi laplace_mpi.c -lm

In [ ]:
!mpirun -n 10 --allow-run-as-root ./laplace_mpi

### Exercise: Add efficiency

Analyze the communication pattern of ```MPI_Send``` and ```MPI_Recv``` in the previous code, how can we make it more efficient? 
Modify the previous code, compile it and run it to see if your modifications result in a faster code.


In [ ]:
%%file laplace_mpi.c

      
}

## Exercise: Kmeans
Bellow you find the sequential code of kmeans (same as the one in OpenMP), what modifications you would do to parallelize it in a distribute system?

In [ ]:
%%file kmeans.cpp
#include <algorithm>
#include <cstdlib>
#include <limits>
#include <random>
#include <vector>

struct Point {
  double x{0}, y{0};
};


using DataFrame = std::vector<Point>;

double squared_l2_distance(Point first, Point second) {
  return std::pow(first.x - second.x,2) + std::pow(first.y - second.y,2);
}


int main (int argc, char* argv[]){ 

  size_t niter = 10;
  size_t k = 2;
  DataFrame data = {Point{1,2},Point{1,2},Point{3,4},Point{10,4},Point{3,4}};
  
// Pick centroids as random points from the dataset.
  static std::random_device seed;
  static std::mt19937 random_number_generator(seed());
  std::uniform_int_distribution<size_t> indices(0, data.size() - 1);

  DataFrame means(k);
  for (auto& cluster : means) {
    cluster = data[indices(random_number_generator)];
  }


  // Find assignments 
  std::vector<size_t> assignments(data.size());
  for (size_t it = 0; it < niter; ++it) {
    for (size_t point = 0; point < data.size(); ++point) {
      
      double min_distance = std::numeric_limits<double>::max();
      size_t best_cluster = 0;
      
      for (size_t cluster = 0; cluster < k; ++cluster) {
        const double distance =
            squared_l2_distance(data[point], means[cluster]);
        if (distance < min_distance) {
          min_distance = distance;
          best_cluster = cluster;
        }
      }

      assignments[point] = best_cluster;
    }

    // Sum up and count points for each cluster.
    double sum_x[k]={0};
    int counts[k]={0};
    double sum_y[k]={0};
    for (size_t point = 0; point < data.size(); ++point) {
      const auto cluster = assignments[point];
      sum_x[cluster] += data[point].x;
      sum_y[cluster] += data[point].y;
      counts[cluster] += 1;
    }
 

    for (size_t cluster = 0; cluster < k; ++cluster) {
      // Turn 0/0 into 0/1 to avoid zero division.
      const auto count = std::max<size_t>(1, counts[cluster]);
      means[cluster].x = sum_x[cluster] / (double) count;
      means[cluster].y = sum_y[cluster] / (double) count;
    }
    

  for (auto& centroid : means) {
    printf("(%f,%f),",centroid.x,centroid.y);
  }
  printf("\n");
}
}

In [ ]:
!mpic++  -o  kmeans_mpi kmeans.cpp

In [ ]:
!mpirun -n 16 --allow-run-as-root ./kmeans_mpi